In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:

!apt-get update
!apt-get install -y wget unzip libvulkan1

!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!apt-get install -f -y

!apt-get install -y xvfb

!pip install selenium chromedriver-autoinstaller

import chromedriver_autoinstaller
chromedriver_autoinstaller.install()

from selenium import webdriver
from selenium.webdriver.chrome.options import Options


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Ign:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,602 kB]
Get:14 http://archive.ubuntu.com/ubunt

In [ ]:
import random
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time
import pandas as pd
from bs4 import BeautifulSoup
import chromedriver_autoinstaller
from google.colab import files
chromedriver_autoinstaller.install()


chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# Disable images
prefs = {
    "profile.managed_default_content_settings.images": 2,
    "profile.managed_default_content_settings.stylesheets": 2,
    "profile.managed_default_content_settings.plugins": 2,
    "profile.managed_default_content_settings.popups": 2,
    "profile.managed_default_content_settings.geolocation": 2,
    "profile.managed_default_content_settings.notifications": 2,
    "profile.managed_default_content_settings.media_stream": 2,
}
chrome_options.add_experimental_option("prefs", prefs)


driver = webdriver.Chrome(options=chrome_options)
driver.set_page_load_timeout(400)


wait = WebDriverWait(driver, 30)

# no all pages load for bt as its also heavy on images. if it doesnt load, try again.
def load_page_with_retry(url, retries=3, delay=5):
    for i in range(retries):
        try:
            driver.get(url)
            print(f"Successfully loaded: {url}")
            return True
        except Exception as e:
            print(f"Attempt {i+1} to load {url} failed: {e}")
            time.sleep(delay)
            delay *= 2
    print(f"Failed to load {url} after {retries} attempts.")
    return False

# testing with ansigtsgenkendelse first
search_url = 'https://www.bt.dk/search?query=ansigtsgenkendelse'

if not load_page_with_retry(search_url, retries=3):
    print("Could not load the page after retries. Exiting.")
    driver.quit()
    exit()

wait = WebDriverWait(driver, 15)

#  Cookies
try:

    cookie_button = wait.until(EC.element_to_be_clickable((By.ID, 'CybotCookiebotDialogBodyLevelButtonLevelOptinAllowAll')))
    cookie_button.click()
    print("Accepted cookies.")
except Exception as e:
    print(f"Could not find or accept cookies: {e}")

time.sleep(2)

# need to scroll to vis flere btn
def click_vis_flere(max_clicks=15):
    click_count = 0
    try:
        while click_count < max_clicks:
            vis_flere_button = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'button.load-more-hits')))
            driver.execute_script("arguments[0].scrollIntoView(true);", vis_flere_button)
            driver.execute_script("arguments[0].click();", vis_flere_button)
            time.sleep(2)
            click_count += 1
            print(f"Clicked 'Vis flere' button {click_count} time(s)")
    except Exception as e:
        print(f"No more 'Vis flere' button found or error occurred: {e}")

click_vis_flere(max_clicks=15)

# when no more vis flere btn, scrape
soup = BeautifulSoup(driver.page_source, 'html.parser')
articles = soup.find_all('article', class_='teaser teaser--flat')

if not articles:
    print("No articles found, please check if the page structure has changed.")
else:
    print(f"Found {len(articles)} articles.")


all_articles = []
MAX_ARTICLES = 200 #testing
article_count = 0


for article in articles:
    if article_count >= MAX_ARTICLES:
        break
    try:
        article_url = article.find('a')['href']
        full_article_url = article_url

        retries = 2
        while retries > 0:
            try:
                driver.get(full_article_url)
                time.sleep(2)

                article_soup = BeautifulSoup(driver.page_source, 'html.parser')

                try:
                    title = article_soup.find('h1').get_text() if article_soup.find('h1') else 'No title'
                except:
                    title = 'No title'

                try:
                    date = article_soup.find('time', class_='timeago')['datetime'] if article_soup.find('time', class_='timeago') else 'No date'
                except:
                    date = 'No date'

                try:
                    category = article_soup.find('a', class_='article-category').get_text() if article_soup.find('a', class_='article-category') else 'No category'
                except:
                    category = 'No category'

                try:
                    content = ""
                    article_body = article_soup.find('main', class_='article-content')

                    if article_body:
                        paragraphs = article_body.find_all('p')
                        for paragraph in paragraphs:
                            content += paragraph.get_text() + " "
                    else:
                        content = 'No content found'
                except:
                    content = 'No content'


                all_articles.append({
                    'url': full_article_url,
                    'title': title,
                    'date': date,
                    'category': category,
                    'content': content.strip()
                })

                article_count += 1
                print(f"Scraped article count: {article_count}")
                break

            except Exception as e:
                retries -= 1
                if retries == 0:
                    print(f"Failed to load article after retries: {full_article_url}, error: {e}")
                    break
                else:
                    print(f"Retrying to load article: {full_article_url}, {retries} retries left.")
                    time.sleep(2)

    except Exception as e:
        print(f"Skipping article: {article_url}, error: {e}")
        continue
    time.sleep(2)

df = pd.DataFrame(all_articles)
df.to_csv('bt_scraped_articles.csv', index=False)
print(df.head())

driver.quit()

Successfully loaded: https://www.bt.dk/search?query=ansigtsgenkendelse
Accepted cookies.
Clicked 'Vis flere' button 1 time(s)
Clicked 'Vis flere' button 2 time(s)
Clicked 'Vis flere' button 3 time(s)
Clicked 'Vis flere' button 4 time(s)
Clicked 'Vis flere' button 5 time(s)
Clicked 'Vis flere' button 6 time(s)
Clicked 'Vis flere' button 7 time(s)
Clicked 'Vis flere' button 8 time(s)
Clicked 'Vis flere' button 9 time(s)
Clicked 'Vis flere' button 10 time(s)
Clicked 'Vis flere' button 11 time(s)
Clicked 'Vis flere' button 12 time(s)
Clicked 'Vis flere' button 13 time(s)
Clicked 'Vis flere' button 14 time(s)
Clicked 'Vis flere' button 15 time(s)
Found 154 articles.
Scraped article count: 1
Scraped article count: 2
Scraped article count: 3
Scraped article count: 4
Scraped article count: 5
Scraped article count: 6
Scraped article count: 7
Scraped article count: 8
Scraped article count: 9
Scraped article count: 10
Scraped article count: 11
Scraped article count: 12
Scraped article count: 13
S

In [ ]:
# Final save of all unique articles to CSV

files.download('bt_scraped_articles.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd
df = pd.read_csv('/content/gdrive/MyDrive/Thesis/Data/bt_scraped_articles.csv')

In [ ]:
df = df.dropna(subset=['content'])
df['date'] = df['date'].str[:10]
df = df.rename(columns={'title': 'header'})

df

,url,header,date,category,content
0,https://www.bt.dk/superligaen/fck-reagerer-paa...,B.T.,2024-09-24,Superligaen,Det gik egentlig strålende på banen - men alli...
1,https://www.bt.dk/samfund/datatilsyn-afkraever...,B.T.,2024-09-09,Samfund,Politiet har for nylig fået tilladelse til at ...
2,https://www.bt.dk/politik/folketinget-tillader...,B.T.,2024-09-05,Politik,"Regeringen og folketingspartier har aftalt, at..."
3,https://www.bt.dk/udland/hollandsk-vagthund-id...,B.T.,2024-09-03,Udland,Amerikansk firma får endnu en trecifret millio...
4,https://www.bt.dk/politik/el-bakker-op-om-brug...,B.T.,2024-08-31,Politik,Brug af ansigtsgenkendelse i sager om farlig b...
...,...,...,...,...,...
147,https://www.bt.dk/tech/facebook-rykker-paa-win...,B.T.,2011-11-02,Tech,Facebook har netop opdateret deres Windows Pho...
148,https://www.bt.dk/danmark/tivoli-oensker-at-sk...,B.T.,2010-04-21,Danmark,For at spare løn til vagter og for at forhindr...
150,https://www.bt.dk/tech/farverigt-kamera,B.T.,2009-08-11,Tech,Let og funktionelt kamera i mange forskellige ...
152,https://www.bt.dk/krimi/politiet-klar-til-at-o...,B.T.,2008-01-27,Krimi,"Politiet vil kunne følge med live, når video-o..."


In [ ]:
from google.colab import files

df.to_csv('bt_scraped_articles_cleaned.csv', index=False)
files.download('bt_scraped_articles_cleaned.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# now we nknow it works, lets do the exact same but for all search terms


chromedriver_autoinstaller.install()


chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

prefs = {
    "profile.managed_default_content_settings.images": 2,
    "profile.managed_default_content_settings.stylesheets": 2,
    "profile.managed_default_content_settings.plugins": 2,
    "profile.managed_default_content_settings.popups": 2,
    "profile.managed_default_content_settings.geolocation": 2,
    "profile.managed_default_content_settings.notifications": 2,
    "profile.managed_default_content_settings.media_stream": 2,
}
chrome_options.add_experimental_option("prefs", prefs)

driver = webdriver.Chrome(options=chrome_options)
driver.set_page_load_timeout(400)

wait = WebDriverWait(driver, 30)

def load_page_with_retry(url, retries=3, delay=5):
    for i in range(retries):
        try:
            driver.get(url)
            print(f"Successfully loaded: {url}")
            return True
        except Exception as e:
            print(f"Attempt {i+1} to load {url} failed: {e}")
            time.sleep(delay)
            delay *= 2
    print(f"Failed to load {url} after {retries} attempts.")
    return False

def click_vis_flere(max_clicks=15):
    click_count = 0
    try:
        while click_count < max_clicks:
            vis_flere_button = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'button.load-more-hits')))
            driver.execute_script("arguments[0].scrollIntoView(true);", vis_flere_button)
            driver.execute_script("arguments[0].click();", vis_flere_button)
            time.sleep(2)
            click_count += 1
            print(f"Clicked 'Vis flere' button {click_count} time(s)")
    except Exception as e:
        print(f"No more 'Vis flere' button found or error occurred: {e}")


search_terms = ['facial recognition', 'face recognition', 'ansigtsaftryk', 'ansigtsscan',
                'masseovervågning ansigt', 'masseovervågning kamera', 'ansigtskamera', 'overvågningssamfund ansigt',
                'overvågningssamfund kamera', 'sikkerhedspakke overvågning', 'tryghedsskabende kamera']

all_articles = []

# go through each search term
for term in search_terms:
    print(f"Scraping for search term: {term}")
    search_url = f'https://www.bt.dk/search?query={term}'

    if not load_page_with_retry(search_url, retries=3):
        print("Could not load the page after retries. Exiting.")
        continue

    try:
        cookie_button = wait.until(EC.element_to_be_clickable((By.ID, 'CybotCookiebotDialogBodyLevelButtonLevelOptinAllowAll')))
        cookie_button.click()
        print("Accepted cookies.")
    except Exception as e:
        print(f"Could not find or accept cookies: {e}")

    time.sleep(2)

    click_vis_flere(max_clicks=15)

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    articles = soup.find_all('article', class_='teaser teaser--flat')

    if not articles:
        print(f"No articles found for search term: {term}. Please check if the page structure has changed.")
        continue
    else:
        print(f"Found {len(articles)} articles for search term: {term}.")
    for article in articles:
        try:
            article_url = article.find('a')['href']
            full_article_url = article_url

            retries = 2
            while retries > 0:
                try:
                    driver.get(full_article_url)
                    time.sleep(2)

                    article_soup = BeautifulSoup(driver.page_source, 'html.parser')

                    try:
                        title = article_soup.find('h1').get_text() if article_soup.find('h1') else 'No title'
                    except:
                        title = 'No title'

                    try:
                        date = article_soup.find('time', class_='timeago')['datetime'] if article_soup.find('time', class_='timeago') else 'No date'
                    except:
                        date = 'No date'

                    try:
                        category = article_soup.find('a', class_='article-category').get_text() if article_soup.find('a', class_='article-category') else 'No category'
                    except:
                        category = 'No category'

                    try:
                        content = ""
                        article_body = article_soup.find('main', class_='article-content')
                        if article_body:
                            paragraphs = article_body.find_all('p')
                            for paragraph in paragraphs:
                                content += paragraph.get_text() + " "
                        else:
                            content = 'No content found'
                    except:
                        content = 'No content'

                    all_articles.append({
                        'url': full_article_url,
                        'title': title,
                        'date': date,
                        'category': category,
                        'content': content.strip(),
                        'search_term': term
                    })

                    break

                except Exception as e:
                    retries -= 1
                    if retries == 0:
                        print(f"Failed to load article after retries: {full_article_url}, error: {e}")
                        break
                    else:
                        print(f"Retrying to load article: {full_article_url}, {retries} retries left.")
                        time.sleep(2)

        except Exception as e:
            print(f"Skipping article: {article_url}, error: {e}")
            continue
        time.sleep(2)

df = pd.DataFrame(all_articles)
df.to_csv('bt_scraped_articles.csv', index=False)
print(df.head())

driver.quit()


Scraping for search term: facial recognition
Successfully loaded: https://www.bt.dk/search?query=facial recognition
Accepted cookies.
No more 'Vis flere' button found or error occurred: Message: 
Stacktrace:
#0 0x5b74f34eeb9a <unknown>
#1 0x5b74f31d4670 <unknown>
#2 0x5b74f3223c48 <unknown>
#3 0x5b74f3223ee1 <unknown>
#4 0x5b74f326a924 <unknown>
#5 0x5b74f3248bad <unknown>
#6 0x5b74f3267dd6 <unknown>
#7 0x5b74f3248923 <unknown>
#8 0x5b74f32166e7 <unknown>
#9 0x5b74f32176de <unknown>
#10 0x5b74f34b866b <unknown>
#11 0x5b74f34bc611 <unknown>
#12 0x5b74f34a44e5 <unknown>
#13 0x5b74f34bd192 <unknown>
#14 0x5b74f34896ef <unknown>
#15 0x5b74f34dd9d8 <unknown>
#16 0x5b74f34ddba7 <unknown>
#17 0x5b74f34ed9ec <unknown>
#18 0x7fd08ed5eac3 <unknown>

Found 6 articles for search term: facial recognition.
Scraping for search term: face recognition
Successfully loaded: https://www.bt.dk/search?query=face recognition
Could not find or accept cookies: Message: 
Stacktrace:
#0 0x5b74f34eeb9a <unknown>


In [ ]:
df

,url,title,date,category,content,search_term
0,https://www.bt.dk/udland/kina-tryner-danske-sv...,B.T.,2020-12-23T12:36:00+01:00,Udland,Ikke meget bliver overladt til tilfældighedern...,facial recognition
1,https://www.bt.dk/udland/nye-uhyggelige-detalj...,B.T.,2018-06-29T19:01:00+02:00,Udland,Torsdag eftermiddag lokal tid trængte en gerni...,facial recognition
2,https://www.bt.dk/udland/nye-uhyggelige-detalj...,B.T.,2018-06-29T18:40:00+02:00,Udland,Torsdag eftermiddag lokal tid trængte en gerni...,facial recognition
3,https://www.bt.dk/digital/facebook-laver-stor-...,B.T.,2017-12-21T06:41:00+01:00,Digital,Facebook vil fremover begynde at sende notifik...,facial recognition
4,https://www.bt.dk/udland/efterforsker-med-over...,B.T.,2017-04-29T08:51:00+02:00,Udland,"Facebook var knap lanceret, da treårige Madele...",facial recognition
5,https://www.bt.dk/erhverv/her-arbejder-de-med-...,B.T.,2016-07-09T11:55:00+02:00,Erhverv,"»Jeg tænkte: Holy Crap«Danske erhvervsfolk, iv...",facial recognition
6,https://www.bt.dk/danmark/dit-privatliv-er-paa...,B.T.,2014-06-19T16:09:00+02:00,Danmark,"Det er svært at svare på, om der foregår noget...",masseovervågning ansigt
7,https://www.bt.dk/krimi/nummerpladen-skal-afsl...,B.T.,2016-05-20T13:24:14+02:00,Krimi,Politibiler får nu installeret automatisk numm...,masseovervågning kamera
8,https://www.bt.dk/krimi/politiet-kan-registrer...,B.T.,2016-05-20T12:31:05+02:00,Krimi,Automatisk nummerpladegenkendelse skal bruges ...,masseovervågning kamera
9,https://www.bt.dk/danmark/politiet-vil-genindf...,B.T.,2015-01-07T23:01:00+01:00,Danmark,Den omstridte lagring af danskernes internetbr...,masseovervågning kamera


In [ ]:

df['date'] = df['date'].str[:10]
df = df.rename(columns={'title': 'header'})

df

,url,header,date,category,content,search_term
0,https://www.bt.dk/udland/kina-tryner-danske-sv...,B.T.,2020-12-23,Udland,Ikke meget bliver overladt til tilfældighedern...,facial recognition
1,https://www.bt.dk/udland/nye-uhyggelige-detalj...,B.T.,2018-06-29,Udland,Torsdag eftermiddag lokal tid trængte en gerni...,facial recognition
2,https://www.bt.dk/udland/nye-uhyggelige-detalj...,B.T.,2018-06-29,Udland,Torsdag eftermiddag lokal tid trængte en gerni...,facial recognition
3,https://www.bt.dk/digital/facebook-laver-stor-...,B.T.,2017-12-21,Digital,Facebook vil fremover begynde at sende notifik...,facial recognition
4,https://www.bt.dk/udland/efterforsker-med-over...,B.T.,2017-04-29,Udland,"Facebook var knap lanceret, da treårige Madele...",facial recognition
5,https://www.bt.dk/erhverv/her-arbejder-de-med-...,B.T.,2016-07-09,Erhverv,"»Jeg tænkte: Holy Crap«Danske erhvervsfolk, iv...",facial recognition
6,https://www.bt.dk/danmark/dit-privatliv-er-paa...,B.T.,2014-06-19,Danmark,"Det er svært at svare på, om der foregår noget...",masseovervågning ansigt
7,https://www.bt.dk/krimi/nummerpladen-skal-afsl...,B.T.,2016-05-20,Krimi,Politibiler får nu installeret automatisk numm...,masseovervågning kamera
8,https://www.bt.dk/krimi/politiet-kan-registrer...,B.T.,2016-05-20,Krimi,Automatisk nummerpladegenkendelse skal bruges ...,masseovervågning kamera
9,https://www.bt.dk/danmark/politiet-vil-genindf...,B.T.,2015-01-07,Danmark,Den omstridte lagring af danskernes internetbr...,masseovervågning kamera


In [ ]:
from google.colab import files

df.to_csv('bt_scraped_articles_otherterms.csv', index=False)
files.download('bt_scraped_articles_otherterms.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd
df = pd.read_csv('/content/gdrive/MyDrive/Thesis/Data/BT/bt_scraped_articles_otherterms.csv')
df1 = pd.read_csv('/content/gdrive/MyDrive/Thesis/Data/BT/bt_scraped_articles_cleaned (1).csv')

In [ ]:
df = pd.concat([df1,df], ignore_index=True)
df['merged_content'] = df['content']
df = df.drop(columns=['content'])
df

,url,header,date,category,search_term,merged_content
0,https://www.bt.dk/superligaen/fck-reagerer-paa...,B.T.,2024-09-24,Superligaen,ansigtsgenkendelse,Det gik egentlig strålende på banen - men alli...
1,https://www.bt.dk/samfund/datatilsyn-afkraever...,B.T.,2024-09-09,Samfund,ansigtsgenkendelse,Politiet har for nylig fået tilladelse til at ...
2,https://www.bt.dk/politik/folketinget-tillader...,B.T.,2024-09-05,Politik,ansigtsgenkendelse,"Regeringen og folketingspartier har aftalt, at..."
3,https://www.bt.dk/udland/hollandsk-vagthund-id...,B.T.,2024-09-03,Udland,ansigtsgenkendelse,Amerikansk firma får endnu en trecifret millio...
4,https://www.bt.dk/politik/el-bakker-op-om-brug...,B.T.,2024-08-31,Politik,ansigtsgenkendelse,Brug af ansigtsgenkendelse i sager om farlig b...
...,...,...,...,...,...,...
175,https://www.bt.dk/politik/stoettepartier-vi-ta...,B.T.,2019-10-08,Politik,sikkerhedspakke overvågning,"R og EL kalder kommende ""sikkerhedspakke"" en g..."
176,https://www.bt.dk/politik/statsminister-bebude...,B.T.,2019-10-08,Politik,sikkerhedspakke overvågning,Politiets og andre myndigheders mulighed for a...
177,https://www.bt.dk/krimi/tusindvis-stalkes-med-...,B.T.,2015-02-13,Krimi,sikkerhedspakke overvågning,"Omkring 4.000 danskere har oplevet, at de er b..."
178,https://www.bt.dk/politik/venstre-vil-have-oek...,B.T.,2024-08-15,Politik,tryghedsskabende kamera,"Venstre mener, at dømte for alvorlig økonomisk..."


In [ ]:
from google.colab import files

df.to_csv('BT.csv', index=False)
files.download('BT.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>